In [ ]:
import yaml
import glob
import os
from pathlib import Path
from typing import Dict, Set, List

# Configure the base path for TO&E files
BASE_PATH = Path("../database/toe")

def load_toe_files() -> Dict[str, dict]:
    """Load all TO&E YAML files and return a dictionary of {id: data}"""
    toe_files = {}
    for yaml_file in BASE_PATH.rglob("*.yaml"):
        with open(yaml_file, encoding='UTF-8') as f:
            data = yaml.safe_load(f)
            if data and 'id' in data:
                toe_files[data['id']] = data
    return toe_files

In [ ]:
def check_circular_references(toe_data: Dict[str, dict]) -> List[str]:
    """Check for circular references in TO&E definitions"""
    def check_unit(unit_id: str, visited: Set[str]) -> bool:
        if unit_id in visited:
            return True
        if unit_id not in toe_data:
            return False
            
        visited.add(unit_id)
        unit = toe_data[unit_id]
        
        if 'subunits' in unit and unit['subunits']:
            for subunit_id in unit['subunits']:
                if check_unit(subunit_id, visited.copy()):
                    return True
        return False
    
    circular_refs = []
    for unit_id in toe_data:
        if check_unit(unit_id, set()):
            circular_refs.append(unit_id)
    return circular_refs

In [ ]:
def find_next_id(prefix: str = "TOEWG") -> str:
    """Find the next available ID number for a given prefix, including gaps in the sequence"""
    toe_data = load_toe_files()
    existing_ids = sorted([int(id.replace(prefix, "")) for id in toe_data.keys() 
                         if id.startswith(prefix)])
    
    if not existing_ids:
        return f"{prefix}000001"
    
    # Find first gap in sequence
    prev_id = 0
    for current_id in existing_ids:
        if current_id - prev_id > 1:
            return f"{prefix}{(prev_id + 1):06d}"
        prev_id = current_id
    
    # If no gaps found, use next number after highest
    return f"{prefix}{(existing_ids[-1] + 1):06d}"

# Example usage
print(f"Next available TOEWG ID: {find_next_id()}")

Next available TOEWG ID: TOEWG000021


In [ ]:
# Load and analyze TO&E data
toe_data = load_toe_files()

print("=== TO&E Database Analysis ===")
print(f"Total units: {len(toe_data)}\n")

print("=== Unit List ===")
for unit_id, unit in toe_data.items():
    print(f"{unit_id}: {unit['name']}")

print("\n=== Circular Reference Check ===")
circular_refs = check_circular_references(toe_data)
if circular_refs:
    print("Warning: Circular references found in:")
    for ref in circular_refs:
        print(f"- {ref}")
else:
    print("No circular references found.")

=== TO&E Database Analysis ===
Total units: 88

=== Unit List ===
TOEWG000017: Schwere Jäger Battalion
TOEWG000006: Panzerbattalion
TOEWG000007: Panzergrenadier Battalion
TOEWG000217: Schwere Jäger Battalion
TOEWG000205: le. Jägerkompanie
TOEWG000204: le. Jägerkompanie Truppe
TOEWG000005: Panzergrenadier Kompanie (Marder)
TOEWG000105: Panzergrenadier Kompanie (MTW)
TOEWG000004: Panzergrenadier Kompanie Truppe (Marder)
TOEWG000104: Panzergrenadier Kompanie Truppe (MTW)
TOEWG000010: Panzer Kompanie
TOEWG000009: Panzer Kompanie Zug
TOEWG000019: Panzermorser Batterie
TOEWG000020: Panzermorser Zug Truppe
TOEWG000220: le. Jägerkompanie
TOEWG000221: Schwere Kompanie Truppe
TOEWG000011: Stabskompanie
TOEWG000015: Engineering Platoon
TOEWG000014: Fernmeldezug
TOEWG000016: Kompanieführungs Zug
TOEWG000201: le. Jägergruppe
TOEWG000203: le. Jäger Zug
TOEWG000202: le. Jäger Zugtruppe
TOEWG000001: Panzergrenadier Gruppe (Marder)
TOEWG000101: Panzergrenadier Gruppe (MTW)
TOEWG000003: Panzergrenadier 